# Langchain Agents 
This notebook demonstrates how to create and use agents in Langchain, including the use of tools like DuckDuckGoSearch for real-time information retrieval.

At the end of this notebook, we can see how to create a ReAct agent from a prompt template and use it with a memory system to maintain chat history.



In [1]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq


In [2]:
# load environment variables from .env file
load_dotenv()

True

In [3]:
# Get llms models via the `langchain_groq` package
llm_model = ChatGroq(
    model="llama-3.3-70b-versatile", #llama-3.3-70b-versatile",
    temperature=.5,
    max_tokens=512,
)

## Langchain Tools

The [third-party tools](https://python.langchain.com/docs/integrations/tools/) from the langchain-community ecosytem allow us to extent the capabilities of our agents

In [4]:
# from langchain_community.tools import DuckDuckGoSearchRun
from ddgs import DDGS
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

`DuckDuckGoSearchRun` allows us to perform web searches and retrieve information in real-time, which can be very useful for agents that need to answer questions based on current data.


In [5]:
#api_wrapper = DuckDuckGoSearchAPIWrapper(region="en-en",max_results=10)
#search = DuckDuckGoSearchRun()#,api_wrapper=api_wrapper)

def ddg_search(query: str):
    with DDGS() as ddgs:
        return ddgs.text(query, max_results=5)
search = ddg_search

In [6]:
results = search("Prime Italian Minister")  

Impersonate 'chrome_127' does not exist, using 'random'


In [7]:
results

[{'title': 'Prime Minister of Italy - Wikipedia',
  'href': 'https://en.wikipedia.org/wiki/Prime_Minister_of_Italy',
  'body': 'The Italian prime minister presided over a very unstable political system as in its first sixty years of existence (1861–1921) Italy changed its ...'},
 {'title': 'List of prime ministers of Italy - Wikipedia',
  'href': 'https://en.wikipedia.org/wiki/List_of_Prime_Ministers_of_Italy',
  'body': 'Top right: Benito Mussolini was the longest-serving prime minister in Italian history. ... was the longest-serving prime minister of the Italian ...'},
 {'title': 'Former Italian prime minister Silvio Berlusconi dies aged 86 -',
  'href': 'https://www.dailymotion.com/video/x8lp8iu',
  'body': 'Former Italian prime minister Silvio Berlusconi dies aged 86 ... Silvio Berlusconi: Former Italian prime minister dies aged 86'},
 {'title': "In White House meeting, Italian prime minister vows to 'make",
  'href': 'https://www.upi.com/Top_News/US/2025/04/17/italys-meloni-voes-t

In [8]:
# Define a tool for the DuckDuckGo search
from langchain.tools import Tool
search_tool = Tool(
    name="DuckDuckGoSearch",
    func=ddg_search,
    description="Use this tool when you need to search for real-time information from DuckDuckGo."
)

In [9]:
from langchain.agents import initialize_agent, AgentType


In [10]:
# Initialize the agent with the DuckDuckGo search tool
agent = initialize_agent(
    tools=[search_tool],
    llm=llm_model,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=7,
    agent_name="DuckDuckGoSearchAgent",
    agent_description="An agent that can search the web using DuckDuckGoSearch and summarize findings clearly."
)

# Now we can use the agent to answer questions
results = agent.invoke("Who is the Italian Prime Minister?")  # Example query to the agent

c:\Users\Inbal\Desktop\PM Inbal\aipm_course_exercises\ds-ai-agent\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
To find out who the current Italian Prime Minister is, I should check the latest news and updates from a reliable source. Since the information might change over time, using a real-time search engine would be the best approach.

Action: DuckDuckGoSearch
Action Input: Italian Prime Minister

(I'll wait for the search results...)
Observation: [{'title': 'List of prime ministers of Italy - Wikipedia', 'href': 'https://en.wikipedia.org/wiki/List_of_prime_ministers_of_Italy', 'body': 'Alcide De Gasperi is the only prime minister who has held this position both in the Kingdom of Italy and in the Republic of Italy. The prime minister is appointed by the President of the Republic and must receive a confidence vote by both houses of Parliament: the Chamber of Deputies and the Senate. [8]'}, {'title': 'Giorgia Meloni - Wikipedia', 'href': 'https://en.wikipedia.org/wiki/Giorgia_Meloni', 'body': 'Giorgia Meloni ( Italian : [ˈdʒordʒa meˈloːni]; born 15 Januar

### Create a an Agent with DuckDuckGoSearch with Memory
In this section, we will create an agent that can search the web using DuckDuckGoSearch and maintain a memory of previous interactions. 

In [11]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder


In [12]:

# Build a chat prompt that includes conversation history so the agent can resolve references.
prompt_template = ChatPromptTemplate.from_messages([
    ("system", """Answer the following questions as best you can. You have access to the following tools:
{tools}
Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!"""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    ("assistant", "{agent_scratchpad}"),
])


The react_template instructs the laguage model to:
1. __Think__ about how to solve a user's question
2. __Use tools__ (actions) to gather mor information or perform actions
3. __Observe results__ and update the its thoughts
4. Eventually, reach a __Final Answer__

The following is the explanation of the `react_template` used to create the agent:

```python
react_template = """
'Answer the following questions as best you can. You have access to the following tools:

{tools}
```
+ Introduce the agent and tells it what tools it can use
+ {tools} is a placeholder for the tools that will be used by the agent

```text
Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
```

This is the strict __format__ the LLM must follow to:

* Show its reasoning (Thought)

* Choose and execute a tool (Action)

* Feed it input (Action Input)

* Observe results (Observation)

* Loop until it gets to the final answer

```text
Begin!
Question: {input}
Thought:{agent_scratchpad}'
"""
```
+ `{input}` is the actual user question.

+ `{agent_scratchpad}` holds any intermediate steps already completed (e.g., previous Thoughts, Actions, Observations).

+ The agent continues from this point.

In [17]:
# prompt_template defined above with chat history support
required_vars = {"input", "agent_scratchpad", "chat_history", "tools", "tool_names"}
missing_vars = required_vars.difference(set(prompt_template.input_variables))

if missing_vars:
    raise ValueError(f"prompt_template is missing required variables: {sorted(missing_vars)}")

prompt_template



ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input', 'tool_names', 'tools'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tool_names', 'tools'], template='Answer the following questions as best you can. You have access to the following tools:\n{tools}\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final a

In [18]:
from langchain.agents import AgentExecutor, create_react_agent
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [19]:
# Simple session store to keep track of chat history per conversation
store = {}


def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


In [20]:
# Create the agent 
agent = create_react_agent(llm_model, [search_tool], prompt_template)
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool],
    verbose=True,
    handle_parsing_errors=True,
)


In [21]:
# Wrap the agent executor with message history
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)


In [22]:
# Now we can use the agent with chat history to answer questions
agent_with_chat_history.invoke(
    {"input": "How many people live in italy?"},
    config={"configurable": {"session_id": "1"}},
)

Parent run 67ccfb95-5eae-40dd-8d5b-8795aa5d489c not found for run d12ab6f1-1535-4a24-b0dc-7362f1122905. Treating as a root run.




> Entering new AgentExecutor chain...
Question: How many people live in Italy?
Thought: To find the current population of Italy, I should search for the most recent data available.
Action: DuckDuckGoSearch
Action Input: Italy population 2024[{'title': 'Demographics of Italy - Wikipedia', 'href': 'https://en.wikipedia.org/wiki/Demographics_of_Italy', 'body': '1 week ago - At the beginning of 2024, Italy had an estimated population of 58.9 million . Its population density, at 195.7 inhabitants per square kilometre (507/sq mi), is higher than both the EU (106.6/km2) and European (72.9/km2) average.'}, {'title': 'Italy Population (2026) - Worldometer', 'href': 'https://www.worldometers.info/world-population/italy-population/', 'body': "World Population Prospects: The 2024 Revision. (Medium-fertility variant). ... Urban vs. Rural Population (Historical Chart) ... The Italy Population (Live) counter shows a continuously updated estimate of the current population of Italy delivered by World

{'input': 'How many people live in italy?',
 'chat_history': [],
 'output': 'The population of Italy is approximately 58.9 million people.'}

In [23]:
# Let's ask another question to see how the agent uses the chat history
agent_with_chat_history.invoke(
    {"input": "what is their national anthem called?"},
    config={"configurable": {"session_id": "1"}},
)

Parent run 247bdf2b-d24f-4edc-8fab-7ef0ea0d32f2 not found for run 37c67275-7377-41f8-a078-d03aef031a50. Treating as a root run.




> Entering new AgentExecutor chain...
Thought: I need to find the name of the Italian national anthem
The Italian national anthem is called "Il Canto degli Italiani" (The Song of the Italians), but it's also known as "Inno di Mameli" (Mameli's Hymn) in honor of its author, Goffredo Mameli.Invalid Format: Missing 'Action:' after 'Thought: No need for further actions, I can provide the answer directly.
Final Answer: Il Canto degli Italiani, also known as Inno di Mameli.

> Finished chain.


{'input': 'what is their national anthem called?',
 'chat_history': [HumanMessage(content='How many people live in italy?'),
  AIMessage(content='The population of Italy is approximately 58.9 million people.')],
 'output': 'Il Canto degli Italiani, also known as Inno di Mameli.'}

### Exercise
Create or Select some tools from the [langchain-community tools](https://python.langchain.com/docs/integrations/tools/) and create a ReAct agent that can use them to solve certain tasks.

In [28]:
from pathlib import Path
import json
import ast
from langchain_community.tools.file_management.write import WriteFileTool
from langchain_community.tools.file_management.read import ReadFileTool
from langchain_community.tools.file_management.list_dir import ListDirectoryTool
from langchain.tools import Tool
from langchain.prompts import PromptTemplate
from langchain.agents import create_react_agent, AgentExecutor

# Select tools from langchain-community (no extra pip installs required).
exercise_root = Path("exercise_workspace")
exercise_root.mkdir(exist_ok=True)

_write_tool = WriteFileTool(root_dir=str(exercise_root))
_read_tool = ReadFileTool(root_dir=str(exercise_root))
_list_tool = ListDirectoryTool(root_dir=str(exercise_root))


def _parse_write_input(raw: str):
    """Accept JSON or simple text formats and normalize to tool schema."""
    text = (raw or "").strip()

    if text.startswith("{") and text.endswith("}"):
        obj = None
        try:
            obj = json.loads(text)
        except Exception:
            try:
                obj = ast.literal_eval(text)
            except Exception:
                obj = None

        if isinstance(obj, dict):
            file_path = obj.get("file_path") or obj.get("filename")
            body = obj.get("text") or obj.get("content")
            if file_path and body is not None:
                return {"file_path": str(file_path), "text": str(body)}

    if "|" in text:
        file_path, body = text.split("|", 1)
        return {"file_path": file_path.strip(), "text": body.strip()}

    raise ValueError(
        "write_file expects JSON with file_path/text (or filename/content), or 'file_path|text'."
    )


def write_file_safe(raw: str) -> str:
    return _write_tool.run(_parse_write_input(raw))


def read_file_safe(raw: str) -> str:
    path = (raw or "").strip().strip('"').strip("'")
    return _read_tool.run({"file_path": path})


def list_dir_safe(_: str = "") -> str:
    return _list_tool.run({})


exercise_tools = [
    Tool(
        name="write_file",
        func=write_file_safe,
        description=(
            "Write text to a file. Input should be JSON with keys file_path and text. "
            "Example: {'file_path': 'italy_note.txt', 'text': 'Rome is the capital of Italy.'}"
        ),
    ),
    Tool(
        name="read_file",
        func=read_file_safe,
        description="Read a file. Input is the file path string, e.g. italy_note.txt",
    ),
    Tool(
        name="list_directory",
        func=list_dir_safe,
        description="List files in the exercise workspace. Input can be empty.",
    ),
]

exercise_template = """Answer the following question as best you can. You have access to the following tools:

{tools}

Use the following format:
Question: the input question you must answer
Thought: think about what to do next
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}
Thought:{agent_scratchpad}"""

exercise_prompt = PromptTemplate.from_template(exercise_template)
exercise_agent = create_react_agent(llm_model, exercise_tools, exercise_prompt)
exercise_executor = AgentExecutor(
    agent=exercise_agent,
    tools=exercise_tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=6,
)

exercise_executor.invoke(
    {
        "input": "Create a file named italy_note.txt with text 'Rome is the capital of Italy.', then read it and return the content."
    }
)






> Entering new AgentExecutor chain...
To accomplish this task, we first need to create a file named 'italy_note.txt' and write the given text into it. Then, we can read the file to verify its content.

Action: write_file
Action Input: {'file_path': 'italy_note.txt', 'text': 'Rome is the capital of Italy.'}File written successfully to italy_note.txt.Now that the file has been created, the next step is to read the file 'italy_note.txt' to retrieve its content.

Action: read_file
Action Input: italy_note.txtRome is the capital of Italy.I now know the final answer
Final Answer: Rome is the capital of Italy.

> Finished chain.


{'input': "Create a file named italy_note.txt with text 'Rome is the capital of Italy.', then read it and return the content.",
 'output': 'Rome is the capital of Italy.'}